<section id="sec1"> </section>
<h1> Imports </h1>

<a href="#back" style="text-decoration: none; color: #333;">Back to table of contents</a>

In [3]:
# Basic imports
import numpy as np
import pandas as pd
import matplotlib as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", message="Parameters: { 'verbose' } are not used.")

##################### sklearn imports 
from scipy.optimize import minimize
from sklearn.metrics import make_scorer
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedStratifiedKFold
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingRegressor
from sklearn.base import clone

##################### Models
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

##################### optuna library import

c:\Users\Nadav\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [7]:
!pip install optuna
!pip install shap
import optuna
import shap
random_state = 42
n_splits = 5


[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
######## Download the data 
#train=pd.read_csv("train_preprocessed.csv")
#test=pd.read_csv("test_preprocessed.csv")

train=pd.read_csv("../data/processed/train_processed.csv")
test=pd.read_csv("../data/processed/test_processed.csv")
sample = pd.read_csv('../data/raw/sample_submission.csv')

train = train.dropna(subset='sii').reset_index().drop('index',axis=1)
train = train.select_dtypes(include='number')
train

NameError: name 'pd' is not defined

In [ ]:
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

In [ ]:
# Function to plot learning curves
def plot_learning_curve(model, X, y, cv):
    train_sizes, train_scores, valid_scores = learning_curve(model, X, y, cv=cv,
                                                             scoring=make_scorer(custom_rmse, greater_is_better=False),
                                                             train_sizes=np.linspace(0.1, 1.0, 5),
                                                             n_jobs=-1)

    train_scores_mean = np.mean(train_scores, axis=1)  # Convert from negative RMSE to RMSE
    valid_scores_mean = np.mean(valid_scores, axis=1)

    plt.figure()
    plt.plot(train_sizes, train_scores_mean, label="Training RMSE")
    plt.plot(train_sizes, valid_scores_mean, label="Cross-Validation RMSE")
    plt.title(f"Learning Curve ({model.__class__.__name__})")
    plt.xlabel("Training Set Size")
    plt.ylabel("RMSE")
    plt.legend(loc="best")
    plt.grid(True)
    plt.show()

# Optuna optimization functions for each model

# CatBoost
def objective_catboost(trial, X_train, y_train):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10),
        'random_strength': trial.suggest_uniform('random_strength', 1, 20),
        'bagging_temperature': trial.suggest_uniform('bagging_temperature', 0, 1),
        'border_count': trial.suggest_int('border_count', 1, 255),
    }

    model = CatBoostRegressor(**params, verbose=0, early_stopping_rounds=50)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, scoring=make_scorer(custom_rmse, greater_is_better=False), cv=cv, n_jobs=-1)

    return np.mean(score)  # Optuna tries to minimize, so we negate RMSE


# XGBoost
def objective_xgboost(trial, X_train, y_train):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10),
    }

    model = XGBRegressor(**params)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, scoring=make_scorer(custom_rmse, greater_is_better=False), cv=cv, n_jobs=-1)

    return np.mean(score)


# LightGBM
def objective_lightgbm(trial, X_train, y_train):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.3),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.5, 1.0),
    }

    model = LGBMRegressor(**params)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, scoring=make_scorer(custom_rmse, greater_is_better=False), cv=cv, n_jobs=-1)

    return np.mean(score)


# RandomForest
def objective_randomforest(trial, X_train, y_train):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 4, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
    }

    model = RandomForestRegressor(**params)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, scoring=make_scorer(custom_rmse, greater_is_better=False), cv=cv, n_jobs=-1)

    return np.mean(score)


# GradientBoosting
def objective_gbr(trial, X_train, y_train):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.3),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
    }

    model = GradientBoostingRegressor(**params)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, scoring=make_scorer(custom_rmse, greater_is_better=False), cv=cv, n_jobs=-1)

    return np.mean(score)

# LinearRegression (no hyperparameters to tune)
def objective_lr(trial, X_train, y_train):
    model = LinearRegression()
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, scoring=make_scorer(custom_rmse, greater_is_better=False), cv=cv, n_jobs=-1)

    return np.mean(score)


In [ ]:
# Function to run hyperparameter tuning and plot learning curve for all models
def tune_model_with_learning_curve(model_name, X_train, y_train):
    if model_name == 'CatBoost':
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective_catboost(trial, X_train, y_train), n_trials=50)

        best_params = study.best_params
        print(f"Best parameters for {model_name}: {best_params}")

        # Train model with best parameters and plot learning curve
        model = CatBoostRegressor(**best_params, verbose=0, early_stopping_rounds=50)
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        plot_learning_curve(model, X_train, y_train, cv)

        return best_params

    elif model_name == 'XGBoost':
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective_xgboost(trial, X_train, y_train), n_trials=50)

        best_params = study.best_params
        print(f"Best parameters for {model_name}: {best_params}")

        model = XGBRegressor(**best_params)
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        plot_learning_curve(model, X_train, y_train, cv)

        return best_params

    elif model_name == 'LightGBM':
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective_lightgbm(trial, X_train, y_train), n_trials=50)

        best_params = study.best_params
        print(f"Best parameters for {model_name}: {best_params}")

        model = LGBMRegressor(**best_params)
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        plot_learning_curve(model, X_train, y_train, cv)

        return best_params

    elif model_name == 'RandomForest':
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective_randomforest(trial, X_train, y_train), n_trials=50)

        best_params = study.best_params
        print(f"Best parameters for {model_name}: {best_params}")

        model = RandomForestRegressor(**best_params)
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        plot_learning_curve(model, X_train, y_train, cv)

        return best_params

    elif model_name == 'GradientBoosting':
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective_gbr(trial, X_train, y_train), n_trials=50)

        best_params = study.best_params
        print(f"Best parameters for {model_name}: {best_params}")

        model = GradientBoostingRegressor(**best_params)
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        plot_learning_curve(model, X_train, y_train, cv)

        return best_params

    elif model_name == 'LinearRegression':
        print("No hyperparameters to tune for Linear Regression.")
        model = LinearRegression()
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        plot_learning_curve(model, X_train, y_train, cv)

        return "No hyperparameters tuned for Linear Regression."